In [ ]:
import json
import unicodedata
import re
import gender_guesser.detector as gender


def cargar_json(ruta):
    with open(ruta, "r", encoding="utf-8") as f:
        return json.load(f)


NOMBRES_INDIGENAS = cargar_json("data/nombres_indigenas.json")
detector = gender.Detector(case_sensitive=False)

def normalizar_nombre(nombre: str) -> str:
    nombre = nombre.strip().lower()
    nombre = unicodedata.normalize("NFD", nombre)
    nombre = nombre.encode("ascii", "ignore").decode("utf-8")
    nombre = re.sub(r"[^a-z\s]", "", nombre)
    return nombre


def obtener_primer_nombre(nombre: str) -> str:
    nombre_normalizado = normalizar_nombre(nombre)
    partes = nombre_normalizado.split()
    return partes[0] if partes else ""


def clasificar_nombre(nombre: str):
    primer_nombre = obtener_primer_nombre(nombre)

    if not primer_nombre:
        return "Desconocido", 0.0, "Sin nombre"

    # Diccionario indígena
    if primer_nombre in NOMBRES_INDIGENAS:
        d = NOMBRES_INDIGENAS[primer_nombre]
        return d["genero"], d["confianza"], "Diccionario indígena"

    # Modelo estadístico
    resultado = detector.get_gender(primer_nombre)

    if resultado == "female":
        return "Mujer", 0.95, "Modelo estadístico"
    elif resultado == "mostly_female":
        return "Mujer", 0.75, "Modelo estadístico"
    elif resultado == "male":
        return "Hombre", 0.95, "Modelo estadístico"
    elif resultado == "mostly_male":
        return "Hombre", 0.75, "Modelo estadístico"
    else:
        return "Desconocido", 0.30, "Nombre ambiguo o poco común"



def calcular_porcentaje_autoras(lista_nombres):
    total = 0
    mujeres = 0
    hombres = 0
    desconocidos = 0

    detalle = []

    for nombre in lista_nombres:
        genero, confianza, fuente = clasificar_nombre(nombre)

        if genero != "Desconocido":
            total += 1

        if genero == "Mujer":
            mujeres += 1
        elif genero == "Hombre":
            hombres += 1
        else:
            desconocidos += 1

        detalle.append({
            "nombre": nombre,
            "genero": genero,
            "confianza": confianza,
            "fuente": fuente
        })

    porcentaje_mujeres = (mujeres / total * 100) if total > 0 else 0

    return {
        "total_autores_validos": total,
        "mujeres": mujeres,
        "hombres": hombres,
        "desconocidos": desconocidos,
        "porcentaje_mujeres": round(porcentaje_mujeres, 2),
        "detalle": detalle
    }

if __name__ == "__main__":

    autores = [
        "Xóchitl Gómez",
        "Citlali Hernández",
        "Maria Jose López",
        "Cuauhtémoc Blanco",
        "Nahuel Pérez",
        "Jordan Smith",
        "Alex Ruiz",
        "Itzel Hernández"
    ]

    resultado = calcular_porcentaje_autoras(autores)

    print("\nRESULTADO GENERAL\n")
    print(f"Total autores válidos: {resultado['total_autores_validos']}")
    print(f"Mujeres: {resultado['mujeres']}")
    print(f"Hombres: {resultado['hombres']}")
    print(f"Desconocidos: {resultado['desconocidos']}")
    print(f"Porcentaje de mujeres autoras: {resultado['porcentaje_mujeres']}%\n")



📊 RESULTADO GENERAL

Total autores válidos: 8
Mujeres: 4
Hombres: 4
Desconocidos: 0
👉 Porcentaje de mujeres autoras: 50.0%

